In [13]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

# データの読み込み
modelling_table = pd.read_csv("preprocessed_data.csv")

# 太陽光発電と風力発電のための別々のモデル
solar_mod = smf.quantreg('Solar_MWh_credit ~ bs(Radiation_dwd,df=5) + bs(Radiation_ncep,df=5)', data=modelling_table)
wind_mod = smf.quantreg('Wind_MWh_credit ~ bs(WindSpeed_dwd,df=8) + bs(WindSpeed_ncep,df=8)', data=modelling_table)

forecast_models_solar = dict()
forecast_models_wind = dict()

# 太陽光エネルギーと風力エネルギーの分位数を別々に計算
for quantile in range(10, 100, 10):
    forecast_models_solar[f"q{quantile}"] = solar_mod.fit(q=quantile/100, max_iter=2500)
    forecast_models_wind[f"q{quantile}"] = wind_mod.fit(q=quantile/100, max_iter=2500)

    modelling_table[f"q{quantile}_solar"] = forecast_models_solar[f"q{quantile}"].predict(modelling_table)
    modelling_table[f"q{quantile}_wind"] = forecast_models_wind[f"q{quantile}"].predict(modelling_table)

    # 発電量は0以上
    modelling_table.loc[modelling_table[f"q{quantile}_solar"] < 0, f"q{quantile}_solar"] = 0
    modelling_table.loc[modelling_table[f"q{quantile}_wind"] < 0, f"q{quantile}_wind"] = 0

    # 小数点3桁で丸める
    modelling_table[f"q{quantile}_solar"] = modelling_table[f"q{quantile}_solar"].round(2)
    modelling_table[f"q{quantile}_wind"] = modelling_table[f"q{quantile}_wind"].round(2)

    # 太陽光と風力の発電量の合計をトータル発電量として計算
    modelling_table[f"q{quantile}"] = modelling_table[f"q{quantile}_solar"] + modelling_table[f"q{quantile}_wind"]

# 太陽光エネルギーと風力エネルギーの予測を分けて保存した更新データを保存
modelling_table.to_csv("analyzed_data_sep.csv", index=False)

# 太陽光エネルギーと風力エネルギーのための分位数回帰モデルをファイルに保存
for quantile in range(10,100,10):
    forecast_models_solar[f"q{quantile}"].save(f"models_sep/solar_model_q{quantile}.pickle")
    forecast_models_wind[f"q{quantile}"].save(f"models_sep/wind_model_q{quantile}.pickle")

/usr/local/lib/python3.11/site-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning: Maximum number of iterations (5000) reached.
  warnings.warn("Maximum number of iterations (" + str(max_iter) +


In [9]:
modelling_table.head()

,ref_datetime,valid_datetime,WindSpeed_dwd,WindSpeed_ncep,Radiation_dwd,Radiation_ncep,Wind_MWh_credit,Solar_MWh_credit,total_generation_MWh,q10_solar,...,q60,q70_solar,q70_wind,q70,q80_solar,q80_wind,q80,q90_solar,q90_wind,q90
0,2020-09-20 00:00:00+00:00,2020-09-20 00:00:00+00:00,11.802604,11.338991,0.0,0.0,498.142,0.0,498.142,0.0,...,514.24,0.0,530.79,530.79,0.0,547.02,547.02,0.0,565.93,565.93
1,2020-09-20 00:00:00+00:00,2020-09-20 00:30:00+00:00,11.648818,11.516161,0.0,0.0,478.788,0.0,478.788,0.0,...,517.16,0.0,533.36,533.36,0.0,549.20,549.20,0.0,567.39,567.39
2,2020-09-20 00:00:00+00:00,2020-09-20 01:00:00+00:00,11.495032,11.693331,0.0,0.0,470.522,0.0,470.522,0.0,...,518.95,0.0,534.88,534.88,0.0,550.50,550.50,0.0,568.24,568.24
3,2020-09-20 00:00:00+00:00,2020-09-20 01:30:00+00:00,11.354128,11.716686,0.0,0.0,482.183,0.0,482.183,0.0,...,515.80,0.0,532.07,532.07,0.0,548.23,548.23,0.0,566.69,566.69
4,2020-09-20 00:00:00+00:00,2020-09-20 02:00:00+00:00,11.213223,11.740043,0.0,0.0,459.216,0.0,459.216,0.0,...,512.26,0.0,528.90,528.90,0.0,545.65,545.65,0.0,564.92,564.92


In [12]:
modelling_table[["valid_datetime","q10","q20","q30","q40","q50","q60","q70","q80","q90","total_generation_MWh"]]

,valid_datetime,q10,q20,q30,q40,q50,q60,q70,q80,q90,total_generation_MWh
0,2020-09-20 00:00:00+00:00,324.31,404.95,448.16,475.58,496.50,514.24,530.79,547.02,565.93,498.142
1,2020-09-20 00:30:00+00:00,327.82,408.20,451.46,478.89,499.74,517.16,533.36,549.20,567.39,478.788
2,2020-09-20 01:00:00+00:00,330.97,410.59,453.68,481.04,501.80,518.95,534.88,550.50,568.24,470.522
3,2020-09-20 01:30:00+00:00,327.80,406.71,449.77,477.33,498.40,515.80,532.07,548.23,566.69,482.183
4,2020-09-20 02:00:00+00:00,324.64,402.61,445.52,473.24,494.60,512.26,528.90,545.65,564.92,459.216
...,...,...,...,...,...,...,...,...,...,...,...
435474,2023-10-22 22:30:00+00:00,3.49,12.05,20.04,28.02,36.56,46.06,57.10,72.51,100.38,2.692
435475,2023-10-22 22:30:00+00:00,3.15,8.53,14.09,20.27,27.18,35.18,44.68,57.92,82.84,2.692
435476,2023-10-22 22:30:00+00:00,7.76,18.13,27.02,35.46,44.10,53.55,64.82,80.10,108.91,2.692
435477,2023-10-22 22:30:00+00:00,4.27,11.55,18.56,25.86,33.72,42.59,53.03,67.42,94.19,2.692
